In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [ ]:
!pip install torch
!pip install transformers
!pip install tqdm

In [ ]:
import json
import random
import torch
import json
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional

from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

In [ ]:
# List of relation types
keys = ['no_relation', 'per:title', 'org:top_members/employees',
        'org:country_of_headquarters', 'per:parents', 'per:age',
        'per:countries_of_residence', 'per:children', 'org:alternate_names',
        'per:charges', 'per:cities_of_residence', 'per:origin', 'org:founded_by',
        'per:employee_of', 'per:siblings', 'per:alternate_names', 'org:website',
        'per:religion', 'per:stateorprovince_of_death', 'org:parents',
        'org:subsidiaries', 'per:other_family', 'per:stateorprovinces_of_residence',
        'org:members', 'per:cause_of_death', 'org:member_of',
        'org:number_of_employees/members', 'per:country_of_birth',
        'org:shareholders', 'org:stateorprovince_of_headquarters',
        'per:city_of_death', 'per:date_of_birth', 'per:spouse',
        'org:city_of_headquarters', 'per:date_of_death', 'per:schools_attended',
        'org:political/religious_affiliation', 'per:country_of_death',
        'org:founded', 'per:stateorprovince_of_birth', 'per:city_of_birth',
        'org:dissolved']

# Assigning indices to the list elements and storing them in a dictionary
rel2id = {key: idx for idx, key in enumerate(keys)}
id2rel = {idx: key for key, idx in rel2id.items()}

# # Printing the dictionaries
# print(rel2id)
# print(id2rel)

In [ ]:
# tell pytorch to use the gpu if available
if torch.cuda.is_available():

    DEVICE = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not
else:
    print('No GPU available, using the CPU instead.')
    DEVICE = torch.device("cpu")

USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    print("using GPU")
else:
    print("using CPU")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

No GPU available, using the CPU instead.
using CPU


In [ ]:
#BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
print('BERT tokenizer loaded')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BERT tokenizer loaded


In [ ]:
"""
By setting the same seed, it ensures that the same sequence of random numbers is generated
each time the program is run, thus making the experimental results reproducible and aiding
in debugging and verifying the robustness of the model.
"""

def setup_seed(seed):
    # Sets the seed for generating random numbers for the CPU.

    # Sets the seed for generating random numbers for all GPUs.
    torch.cuda.manual_seed_all(seed)

    # Sets the seed for generating random numbers with NumPy.
    np.random.seed(seed)

    # Sets the seed for the built-in Python random module.
    random.seed(seed)

setup_seed(44)

In [ ]:
def load_tacred_dataset(file_path):
    """
    Load the TACRED dataset from a JSON file.

    Args:
    file_path (str): The path to the JSON file containing the dataset.

    Returns:
    dict: The loaded dataset.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

Our data preprocessing function `prepare_data(data)` plays a key role in natural language processing tasks. This function takes raw data and converts it into an easy-to-process format. It goes through each data instance, extracts the start and end index of the entity, extracts the entity text from it, and then gets the relationship label. It then combines words from the original text into sentences and organizes this processed information into dictionary form. Ultimately, it adds the dictionary of each data instance to a list and returns the list as output. This function not only provides the function of data format conversion, but also facilitates subsequent model training and text analysis.

In [ ]:
def prepare_data(data):
    # Get the number of instances in the data
    n = len(data)

    # Initialize an empty list to store processed data
    info = []

    # Iterate through each instance in the data
    for i in range(n):
        # Initialize an empty dictionary for storing processed information of a single data instance
        single_data = {}

        # Extract start and end indices of subject and object entities
        ss = data[i]["subj_start"]
        se = data[i]["subj_end"]
        os = data[i]["obj_start"]
        oe = data[i]["obj_end"]

        # Extract subject and object tokens based on their start and end indices
        subj = data[i]['token'][ss: se+1]
        obj = data[i]['token'][os: oe+1]

        # Convert subject and object tokens into strings
        ent1 = ' '.join(subj)
        ent2 = ' '.join(obj)

        # Extract relation label
        rel = data[i]["relation"]

        # Concatenate all tokens to form the original sentence
        text = " ".join(data[i]["token"])

        # Store processed information in the dictionary
        single_data['rel'] = rel
        single_data['ent1'] = ent1
        single_data['ent2'] = ent2
        single_data['text'] = text

        # Append processed information of a single data instance to the list
        info.append(single_data)

    # Return the processed data
    return info

The `process_relation_extraction_data(info, max_length=64)` function plays a vital role in the relationship extraction task. This function is used to process raw data and prepare it into a format suitable for model input. First, the function traverses the provided data information and integrates entities, text information, and relationship tags. For each data instance, the function will splice the entity and text information into a sentence, and perform word segmentation and filling operations. Next, the sentences are tokenized by adding special tags. The function truncates or pads sentences according to the given maximum length to ensure consistent sentence lengths. Finally, the function adds the processed sentences and their corresponding attention masks to the output data. In this way, we are able to transform the raw data into a form suitable for model training and prepare it for further relationship extraction tasks.

In [ ]:
def process_relation_extraction_data(info, max_length=64):
    # Initialize a dictionary to store processed data
    data = {}
    data['label'] = []  # List to store relation labels
    data['mask'] = []   # List to store attention masks
    data['text'] = []   # List to store tokenized and padded texts

    # Iterate through each instance in the provided 'info' data
    for line in info:
        # Check if the relation label is present in the 'rel2id' dictionary
        if line['rel'] not in rel2id:
            # If the relation label is not found, assign label 0 (for unknown relation)
            data['label'].append(0)
        else:
            # If the relation label is found, assign its corresponding index
            data['label'].append(rel2id[line['rel']])

        # Concatenate subject, object, and text to form a single sentence
        sent = line['ent1'] + line['ent2'] + line['text']

        # Tokenize the concatenated sentence and add special tokens
        indexed_tokens = tokenizer.encode(sent, add_special_tokens=True)

        # Determine the available length of the tokenized sentence
        avai_len = len(indexed_tokens)

        # Pad the tokenized sentence with 0s to match the maximum length
        while len(indexed_tokens) < max_length:
            indexed_tokens.append(0)

        # Trim the tokenized sentence to the maximum length
        indexed_tokens = indexed_tokens[:max_length]

        # Convert the tokenized sentence to a PyTorch tensor
        indexed_tokens = torch.tensor(indexed_tokens).long().unsqueeze(0)  # (1, L)

        # Create an attention mask for the tokenized sentence
        att_mask = torch.zeros(indexed_tokens.size()).long()  # (1, L)
        att_mask[0, :avai_len] = 1

        # Append the tokenized and padded sentence, and its attention mask to the data dictionary
        data['text'].append(indexed_tokens)
        data['mask'].append(att_mask)

    return data

In [ ]:
def convert_data_to_tensors(data):
    # Extract text, mask, and label from the data
    text = data['text']
    mask = data['mask']
    label = data['label']

    # Convert text and mask tensors to numpy arrays
    text = [t.numpy() for t in text]
    mask = [t.numpy() for t in mask]

    # Convert numpy arrays to PyTorch tensors
    text = torch.tensor(text)
    mask = torch.tensor(mask)
    label = torch.tensor(label)

    return text, mask, label

The `BERT_Classifier` class implements a BERT-based classifier for relation extraction tasks. The model utilizes pre-trained BERT encoder to extract text features and perform classification through fully connected layers. During the initialization process, the pretrained BERT model is loaded by calling the `from_pretrained` method, and a fully connected layer containing `label_num` output categories is defined. The model also includes a dropout layer to prevent overfitting and uses cross-entropy loss as the loss function for classification.

In the forward propagation process, the input data passes through the BERT encoder to obtain text feature representation, and then is classified through the fully connected layer. The final output is the class probability distribution predicted by the model. If a label is provided, the model computes the cross-entropy loss and returns the loss value and the predicted probability distribution; otherwise, only the predicted probability distribution is returned.

When in use, we can initialize the model according to specific tasks and number of labels, and move it to a designated computing device (such as GPU). Use `labels_num=len(rel2id)` to get the number of relationship labels and create the corresponding model instance.

In [ ]:
class BERT_Classifier(nn.Module):
    def __init__(self, label_num):
        super().__init__()
        # Initialize the BERT encoder from pre-trained weights
        self.encoder = BertModel.from_pretrained('bert-base-uncased')
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.1, inplace=False)
        # Fully connected layer for classification
        self.fc = nn.Linear(768, label_num)  # 768 is the hidden size of BERT
        # Cross-entropy loss criterion
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, label=None):
        # Pass the input through the BERT encoder
        x = self.encoder(x, attention_mask=attention_mask)[0]  # Output is tuple (last_hidden_state, pooler_output), we take the last_hidden_state
        # Take only the first token's output (CLS token)
        x = x[:, 0, :]
        # Apply dropout
        x = self.dropout(x)
        # Pass through the fully connected layer
        x = self.fc(x)
        # If label is not provided, return logits only
        if label is None:
            return None, x
        else:
            # Calculate the cross-entropy loss and return both loss and logits
            return self.criterion(x, label), x

labels_num=len(rel2id)
model = BERT_Classifier(labels_num)
model.to(device)

BERT_Classifier(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

The `train` function is an important tool for model training. It receives a neural network model, a training data set, a validation data set, and some hyperparameters such as the number of training epochs, learning rate, and batch size. During training, this function sets the model to training mode and uses stochastic gradient descent (SGD) as the optimizer. It iterates over the training data set, performing forward propagation, computing loss, back propagation, and parameter updates in each batch. At the same time, it monitors the loss and accuracy of the model in real time, and uses the tqdm library to display the training progress. At the end of each cycle, this function calculates the accuracy, loss, F1 score and other evaluation indicators of the model on the training set and validation set, and prints out these indicators.

Additionally, at the end of each epoch, the function calls the `eval` function to evaluate the model on the validation set. After training is completed, the model status will be saved to a .pth file for subsequent model application or further training. Overall, the `train` function is a core tool for completing model training and monitoring performance.

In the `train` function, in addition to monitoring the loss and accuracy of the model, the F1 score is also used as an evaluation metric for model performance. At the end of each training cycle, the model's accuracy, loss, and F1 score under three different weighting methods on the training set and validation set were calculated: macro average (macro), micro average (micro), and weighted average (weighted ).

The F1 score is an indicator that combines the precision and recall of the model. It comprehensively evaluates the imbalanced class distribution and classifier effect. It is calculated as the harmonic mean of precision and recall, which can be used to evaluate the performance of the model in multi-classification tasks. During this training process, three different ways of calculating F1 scores were used to comprehensively evaluate the model's performance on different categories.

Specifically, when calculating these evaluation indicators, the `f1_score` function is used, in which the `average` parameter is set to `'macro'`, `'micro'` and `'weighted'` respectively to calculate the macro average, micro Average and weighted average F1 scores. These F1 scores provide an assessment of model performance at different levels, helping to better understand how the model performs in classification tasks.

In [ ]:
def train(net, train_dataset, dev_dataset, num_epochs, learning_rate, batch_size):

    # Set the network to training mode
    net.train()

    # Define the optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0)

    # Create a data loader for training data
    train_iter = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

    # Lists to store metrics
    epoch_losses = []
    epoch_accuracies = []

    for epoch in range(num_epochs):

        print('Training...')

        correct = 0
        total = 0
        iter = 0
        all_pred = []
        all_true = []

        # Initialize tqdm to show progress bar
        progress_bar = tqdm(train_iter, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')

        for text, mask, y in progress_bar:
            iter += 1
            optimizer.zero_grad()

            # If the batch size is not equal to the specified batch size, break the loop
            if text.size(0) != batch_size:
                break

            # Reshape text and mask tensors
            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            # Move tensors to GPU if available
            if USE_CUDA:
                text = text.cuda()
                mask = mask.cuda()
                y = y.cuda()

            # Forward pass
            loss, logits = net(text, mask, y)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute accuracy
            _, predicted = torch.max(logits.data, 1)
            total += text.size(0)
            correct += predicted.data.eq(y.data).cpu().sum()

            # Collect predictions and true labels
            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item(), 'accuracy': correct.item() / total})

        # After the end of each epoch, compute metrics
        accuracy = correct.cpu().numpy().tolist()/total
        loss = loss.detach().cpu()

        epoch_losses.append(loss)
        epoch_accuracies.append(accuracy)

        # Compute F1 scores
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')

        # Print metrics
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Loss: {loss.mean().numpy().tolist()}, Accuracy: {accuracy}")
        print(f"Macro F1: {macro_f1}, Micro F1: {micro_f1}, Weighted F1: {weighted_f1}")

        print("Validation...")

        dev_acc, dev_macro_f1, dev_micro_f1, dev_weighted_f1, precision_list, recall_list= eval(net, dev_dataset, batch_size)

        print()

    # torch.save(accuracy, str(accuracy)+'.pth')

    torch.save(model.state_dict(), 'BERT_EX1_80_model.pth')

    return epoch_losses, epoch_accuracies


In [ ]:
def eval(net, dataset, batch_size):
    # Set the network to evaluation mode
    net.eval()

    # Create an iterator for the evaluation dataset
    eval_iter = DataLoader(dataset, batch_size, shuffle=False)

    # Lists to store predictions and true labels
    all_pred = []
    all_true = []

    # Lists to store evaluation metrics
    acc_list = []
    macro_f1_list = []
    micro_f1_list = []
    weighted_f1_list = []
    precision_list = []  # List to store precision scores
    recall_list = []     # List to store recall scores

    with torch.no_grad():
        correct = 0  # Counter for correctly classified samples
        total = 0    # Counter for total samples
        # Progress bar for visualization during evaluation
        progress_bar = tqdm(eval_iter, desc='Evaluation', unit='batch')

        # Iterate through each batch in the evaluation dataset
        for text, mask, y in progress_bar:
            # If the batch size is not as expected, break the loop
            if text.size(0) != batch_size:
                break

            # Reshape text and mask tensors
            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            # Move tensors to GPU if available
            if USE_CUDA:
                text, mask, y = text.cuda(), mask.cuda(), y.cuda()

            # Forward pass through the network
            outputs = net(text, mask)
            # Unpack outputs if it's a tuple (contains both loss and logits)
            loss, logits = outputs if isinstance(outputs, tuple) else (None, outputs)

            # Calculate predicted labels
            _, predicted = torch.max(logits, 1)
            # Update total and correct counts
            total += y.size(0)
            correct += (predicted == y).sum().item()

            # Store predicted and true labels for computing F1 score later
            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

            # Calculate accuracy for the current batch
            accuracy = correct / total
            progress_bar.set_postfix({'accuracy': accuracy})

        # Calculate overall accuracy
        acc = correct / total
        # Calculate F1 scores
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')
        # Calculate precision and recall
        precision = precision_score(all_true, all_pred, average='macro')
        recall = recall_score(all_true, all_pred, average='macro')

        # Append metrics to respective lists
        acc_list.append(acc)
        macro_f1_list.append(macro_f1)
        micro_f1_list.append(micro_f1)
        weighted_f1_list.append(weighted_f1)
        precision_list.append(precision)
        recall_list.append(recall)

        # Print evaluation results
        print(f"Eval Result: right {correct}, total {total}, Acc: {acc:.4f}")
        print(f"Macro F1: {macro_f1:.4f}, Micro F1: {micro_f1:.4f}, Weighted F1: {weighted_f1:.4f}")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")

        # Return evaluation metrics lists
        return acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list


In [ ]:
# Load data from JSON file
train_data = load_tacred_dataset('/home/featurize/data/different_size_dataset/80/train_80%.json')
dev_data = load_tacred_dataset('/home/featurize/data/TACRED/dev.json')
test_data = load_tacred_dataset('/home/featurize/data/TACRED/test.json')

In [ ]:
# ent1+ent2+text dataset
train_info = prepare_data(train_data)
dev_info = prepare_data(dev_data)
test_info = prepare_data(test_data)

print(len(train_info))
print(len(dev_info))
print(len(test_info))

54496
22631
15509


In [ ]:
train_data = process_relation_extraction_data(train_info, 128)
dev_data = process_relation_extraction_data(dev_info, 128)
test_data = process_relation_extraction_data(test_info, 128)

print(len(train_data['label']))
print(len(dev_data['label']))
print(len(test_data['label']))

54496
22631
15509


In [ ]:
# Preprocess train data
train_text, train_mask, train_label = convert_data_to_tensors(train_data)
print("--train data--")
print(train_text.shape)
print(train_mask.shape)
print(train_label.shape)

# Preprocess dev data
dev_text, dev_mask, dev_label = convert_data_to_tensors(dev_data)
print("--eval data--")
print(dev_text.shape)
print(dev_mask.shape)
print(dev_label.shape)

# Preprocess test data
test_text, test_mask, test_label = convert_data_to_tensors(test_data)
print("--test data--")
print(test_text.shape)
print(test_mask.shape)
print(test_label.shape)

/tmp/ipykernel_34494/307683292.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  text = torch.tensor(text)


--train data--
torch.Size([54496, 1, 128])
torch.Size([54496, 1, 128])
torch.Size([54496])
--eval data--
torch.Size([22631, 1, 128])
torch.Size([22631, 1, 128])
torch.Size([22631])
--test data--
torch.Size([15509, 1, 128])
torch.Size([15509, 1, 128])
torch.Size([15509])


In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_text,train_mask,train_label)
dev_dataset = torch.utils.data.TensorDataset(dev_text,dev_mask,dev_label)

In [ ]:
epoch_losses, epoch_accuracies = train(model, train_dataset, dev_dataset, 20, 0.002, 16)

Training...


Epoch 1/20: 100%|██████████| 3406/3406 [02:14<00:00, 25.38batch/s, loss=0.267, accuracy=0.811] 


Epoch 1/20
Loss: 0.26725372672080994, Accuracy: 0.8111054022313564
Macro F1: 0.03308598427757634, Micro F1: 0.8111054022313564, Weighted F1: 0.7377103624285543
Validation...


Evaluation: 100%|█████████▉| 1414/1415 [00:20<00:00, 70.26batch/s, accuracy=0.774]
/environment/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Eval Result: right 17512, total 22624, Acc: 0.7740
Macro F1: 0.0505, Micro F1: 0.7740, Weighted F1: 0.6996
Precision: 0.0795, Recall: 0.0543

Training...


...

Epoch 12/20: 100%|██████████| 3406/3406 [02:12<00:00, 25.80batch/s, loss=0.00839, accuracy=0.99]  


Epoch 12/20
Loss: 0.008385044522583485, Accuracy: 0.9899992660011744
Macro F1: 0.9173365926900269, Micro F1: 0.9899992660011744, Weighted F1: 0.9898576649934752
Validation...


Evaluation: 100%|█████████▉| 1414/1415 [00:21<00:00, 67.09batch/s, accuracy=0.804]
/environment/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Eval Result: right 18191, total 22624, Acc: 0.8041
Macro F1: 0.3999, Micro F1: 0.8041, Weighted F1: 0.7948
Precision: 0.4481, Recall: 0.3785

Training...


Epoch 13/20:  95%|█████████▍| 3219/3406 [02:05<00:07, 25.67batch/s, loss=0.096, accuracy=0.992]   

In [ ]:
import matplotlib.pyplot as plt

num_epochs = 20

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), epoch_losses, label='Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), epoch_accuracies, label='Accuracy', color='orange')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
test_dataset = torch.utils.data.TensorDataset(test_text, test_mask, test_label)
acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list = eval(model, test_dataset, 16)

In [ ]:
print(acc_list)
print(precision_list)
print(recall_list)
print()
print(macro_f1_list)
print(micro_f1_list)
print(weighted_f1_list)